# Decision Trees
In this exercise I will explore how a decision tree is splitted using [Information Gain](https://en.wikipedia.org/wiki/Information_gain_(decision_tree)).

In Decision Tree, we decide if a node will be split or not by looking at the **Information Gain** that split would give us.

$$ Information \; Gain = H(p_1^{node}) - \left( w^{left} H(p_1^{left}) + w^{right} H(p_1^{right}) \right)$$

Where $H$ is the entropy, defined as:

$$ H(p_1) = -p_1 \log_2 (p_1) - (1-p_1) \log_2 (1-p_1) $$

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

The data I will use is the following:

|               | Ear Shape | Face Shape | Whiskers|   Cat |
|:-------------:|:---------:|:----------:|:-------:|:-----:|
|<img src="images/0.png" alt="drawing" width="50"> | Pointy | Round     | Present | 1 |
|<img src="images/1.png" alt="drawing" width="50"> | Floppy | Not Round | Present | 1 |
|<img src="images/2.png" alt="drawing" width="50"> | Floppy | Round     | Absent  | 0 |
|<img src="images/3.png" alt="drawing" width="50"> | Pointy | Not Round | Present | 0 |
|<img src="images/4.png" alt="drawing" width="50"> | Pointy | Round     | Present | 1 |
|<img src="images/5.png" alt="drawing" width="50"> | Pointy | Round     | Absent  | 1 |
|<img src="images/6.png" alt="drawing" width="50"> | Floppy | Not Round | Absent  | 0 |
|<img src="images/7.png" alt="drawing" width="50"> | Pointy | Round     | Absent  | 1 |
|<img src="images/8.png" alt="drawing" width="50"> | Floppy | Round     | Absent  | 0 |
|<img src="images/9.png" alt="drawing" width="50"> | Floppy | Round     | Absent  | 0 |

I will use **one-hot encoding** to encode the categorical features.
- Ear Shape: Pointy = 1, Floppy = 0
- Face Shape: Round = 1, Not Round = 0
- Whiskers: Present = 1, Absent = 0

With this election, the dataset is:

In [13]:
# Data set
X_train = np.array([[1,1,1],
                   [0,0,1],
                   [0,1,0],
                   [1,0,1],
                   [1,1,1],
                   [1,1,0],
                   [0,0,0],
                   [1,1,0],
                   [0,1,0],
                   [0,1,0]])

y_train = np.array([1,1,0,0,1,1,0,1,0,0])

On each node, we compute the gain information for each feature, the split the node on the feature with the higher information gain, by comparing the entropy of the node wjth the weighted entropy in the two splitted nodes.

Let's write a function to compute the entropy:

In [ ]:
def entropy(p):
    if p == 0 or p ==1:
        return 0
    else:
        H = -p * np.log2(p) - (1 - p)*np.log2(1 - p)
        return H

In [9]:
print(entropy(0.5))

1.0


In [17]:
def split_indices(X, index_features):
    """
        Given a dataset and a index feature, return two lists for the two split nodes, the left node has the animals that have 
        that feature = 1 and the right node those that have the feature = 0 
        index feature = 0 => ear shape
        index feature = 1 => face shape
        index feature = 2 => whiskers
    """

    left_indices = []
    right_indices = []
    
    for i, x in enumerate(X):
        if x[index_features] == 1:
            left_indices.append(i)
        else:
            right_indices.append(i)
    return left_indices, right_indices

In [18]:
split_indices(X_train, 0)

([0, 3, 4, 5, 7], [1, 2, 6, 8, 9])

If we see the table of values, we see that those values equal to 1 for the *ear shape* feature, second column, are returned in the **left_indices** list while the others are in the **right_indices** list.

Now, we need another function to compute the weighted entropy in the splitted node.

In [22]:
def weighted_entropy(X, y, left_indices, right_indices):
    """
            This function takes the splitted dataset, the indices we chose to split and returns the weighted entropy.
    """

    w_left = len(left_indices) / len(X)                         # Computes the proportion of animals in each node.
    w_right = len(right_indices) / len(X)
    p_left = sum(y[left_indices]) / len(left_indices)           # Computes the proportion of cats in each split.
    p_right = sum(y[right_indices]) / ( len(right_indices))

    weighted_entropy = w_left * entropy(p_left) + w_right * entropy(p_right)

    return weighted_entropy

In [23]:
left_indices, right_indices = split_indices(X_train, 0)
weighted_entropy(X_train, y_train, left_indices, right_indices)

0.7219280948873623

So, the weighted entropy in the 2 split nodes is $\approx 0.72$. To computhe the **Information Gain ** we must substract it from the entropy in the node we chose to split (in this case the root node).

In [24]:
def information_gain(X, y, left_indices, right_indices):
    """
            Here, X has the elements in the node and y is theirs respectives classes
    """

    p_node = sum(y) / len(y)
    h_node = entropy(p_node)
    w_entropy = weighted_entropy(X, y, left_indices, right_indices)
    return h_node - w_entropy

In [25]:
information_gain(X_train, y_train, left_indices, right_indices)

0.2780719051126377

Now, let's compute the information gain if we split the root node for each feature:

In [26]:
for i, feature_name in enumerate(['Ear Shape', 'Face Shape', 'Whiskers']):
    left_indices, right_indices = split_indices(X_train, i)
    i_gain = information_gain(X_train, y_train, left_indices, right_indices)
    print(f"Feature: {feature_name}, information gain if we split the root using this feature: {i_gain:.2f}")

Feature: Ear Shape, information gain if we split the root using this feature: 0.28
Feature: Face Shape, information gain if we split the root using this feature: 0.03
Feature: Whiskers, information gain if we split the root using this feature: 0.12


So, the best feature to split is indeed the **Ear Shape**.

The process is **recursive**, which means we must perform these calculations for each node until we meet a stopping criteria:

- If the tree depth after splitting exceeds a threshold
- If the resulting node has only 1 class
- If the information gain of splitting is below a threshold